In [56]:
import pickle
import time
from typing import List

import numpy as np

import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analyzer import data_analyzer, physicality_violation_check
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_z0_1q, get_z1_1q, get_x0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import ProjectedLinearEstimator

In [2]:
%load_ext autoreload
%autoreload 2

In [6]:
def estimate_sequence(name: str, true_object: State, num_data: List[int], iterations: int, on_para_eq_constraint: bool=True):
    qst = StandardQst(povms, on_para_eq_constraint=on_para_eq_constraint)

    # generate empi dists and calc estimate
    obj_sequences = []
    start = time.time()
    for iteration in tqdm(range(iterations)):
        empi_dists_seq = qst.generate_empi_dists_sequence(
            true_object, num_data
        )

        estimator = LinearEstimator()
        #obj_sequence = estimator.calc_estimate_sequence_qoperation(qst, empi_dists_seq)
        obj_sequence = estimator.calc_estimate_sequence(qst, empi_dists_seq)

        # info = {
        #     "iteration": iteration,
        #     "empi_dists_seq": empi_dists_seq,
        #     "obj_sequence": obj_sequence.estimated_var_sequence,
        # }
        # print(info)
        obj_sequences.append(obj_sequence.estimated_qoperation_sequence)

    end = time.time()
    print(f"time(s)={end - start}")
    #with open(f"state_obj_sequences_{name}_{on_para_eq_constraint}_{iterations}.pkl", "wb") as f:
    #    pickle.dump(obj_sequences, f)

    #with open("obj_sequences.pkl", "rb") as f:
    #    obj_sequences = pickle.load(f)
    return obj_sequences

In [18]:
def estimate(name: str, true_object: State, num_data: List[int], iterations: int, on_para_eq_constraint: bool=True):
    qst = StandardQst(povms, on_para_eq_constraint=on_para_eq_constraint)

    # generate empi dists and calc estimate
    obj_sequences = []
    start = time.time()
    for iteration in tqdm(range(iterations)):
        empi_dists_seq = qst.generate_empi_dists_sequence(
            true_object, num_data
        )

        estimator = LinearEstimator()
        obj_sequence = estimator.calc_estimate_sequence(qst, empi_dists_seq)
        obj_sequences.append(obj_sequence.estimated_qoperation_sequence)

    end = time.time()
    print(f"time(s)={end - start}")
    return obj_sequences

In [19]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
povms = [povm_x, povm_y, povm_z]

In [64]:
true_object = get_z0_1q(c_sys)
num_data = [1000]
iterations = 1000

param_affine_est_linear = estimate("z0", true_object, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate("z0", true_object, num_data, iterations, on_para_eq_constraint=False)

param_affine_est_linear = [p[0] for p in param_affine_est_linear]
param_linear_est_linear = [p[0] for p in param_linear_est_linear]

  1%|          | 9/1000 [00:00<00:11, 87.08it/s]

time(s)=11.32900881767273


100%|██████████| 1000/1000 [00:15<00:00, 66.42it/s]


time(s)=15.057640075683594


# Physicality Violation

In [65]:
true_object.vec

array([0.70710678, 0.        , 0.        , 0.70710678])

## on_para_eq_constraint = True

In [66]:
violation_result = physicality_violation_check.get_physicality_violation_result_for_state(param_affine_est_linear)
violation_result.keys()

dict_keys(['violation_list'])

In [67]:
physicality_violation_check.make_prob_dist_histogram(violation_result["violation_list"], bin_size=0.01)

## on_para_eq_constraint = False

In [131]:
eig_list = physicality_violation_check.get_sorted_eigenvalues_list(param_linear_est_linear)

100%|██████████| 1000/1000 [00:00<00:00, 4533.09it/s]


In [133]:
a, b = physicality_violation_check.get_sum_of_eigenvalues_violation(eig_list)

[1.0001459786902216, -0.00014597869022222324]
384: [1.0, -2.220446049250313e-16]


In [128]:
eig_list[999]

[1.0001459786902216, -0.00014597869022222324]

In [134]:
for i, values in enumerate(violation_result["sorted_eigenvalues_list"]):
    fig = physicality_violation_check.make_prob_dist_histogram(values, bin_size=0.0001)
    fig.update_layout(title=f"N={num_data[0]}, Nrep={len(values)}")
    fig.update_xaxes( title=f"Eigenvalue (i={i})")
    fig.show()

In [135]:
value_list = violation_result["sum_of_eigenvalues"]["less_than_zero"]
fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.0001, annotation_vlines=[0])

# TODO: iterationsとnum_dataを与える方法
n_rep = len(violation_result["sorted_eigenvalues_list"][0])
n = num_data[0]
title = f"N={n}, Nrep={n_rep}, Number of Unphysical estimates={len(value_list)}"
fig.update_layout(title=title)
fig.update_xaxes( title=f"Sum of unphysical eigenvalues (<0)")
fig.show()

value_list = violation_result["sum_of_eigenvalues"]["greater_than_one"]
fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.0001, annotation_vlines=[1])

n_rep = len(violation_result["sorted_eigenvalues_list"][0])
n = num_data[0]
title = f"N={n}, Nrep={n_rep}, Number of Unphysical estimates={len(value_list)}"
fig.update_layout(title=title)  # TODO
fig.update_xaxes( title=f"Sum of unphysical eigenvalues (>1)")
fig.show()

# True Object = State([ 1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6) ])

In [33]:
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

vec = np.array([1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],dtype=np.float64)
true_object = State(c_sys, vec)
num_data = [1000]  # N
iterations = 1000  # Nrep

In [34]:
povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
povms = [povm_x, povm_y, povm_z]

In [39]:
name = "1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)"
param_affine_est_linear = estimate(name, true_object, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate(name, true_object, num_data, iterations, on_para_eq_constraint=False)

param_affine_est_linear = [p[0] for p in param_affine_est_linear]
param_linear_est_linear = [p[0] for p in param_linear_est_linear]

  1%|          | 10/1000 [00:00<00:10, 94.46it/s]

time(s)=10.005339860916138


100%|██████████| 1000/1000 [00:09<00:00, 104.74it/s]

time(s)=9.552195072174072


## on_para_eq_constraint = True

In [42]:
violation_result = physicality_violation_check.get_physicality_violation_result_for_state(param_affine_est_linear)
violation_result.keys()

dict_keys(['violation_list'])

In [43]:
value_list = violation_result["violation_list"]
physicality_violation_check.make_prob_dist_histogram(value_list, x_range=(-1, 2), bin_size=0.01)

## on_para_eq_constraint = False

In [44]:
violation_result = physicality_violation_check.get_physicality_violation_result_for_state(param_linear_est_linear)
violation_result.keys()

100%|██████████| 1000/1000 [00:00<00:00, 7085.40it/s]


dict_keys(['sorted_eigenvalues_list', 'sum_of_eigenvalues'])

In [45]:
for i, values in enumerate(violation_result["sorted_eigenvalues_list"]):
    fig = physicality_violation_check.make_prob_dist_histogram(values, bin_size=0.0001)
    fig.update_layout(title=f"Nrep={len(values)}")
    fig.update_xaxes( title=f"Eigenvalue (i={i})")
    fig.show()

In [46]:
value_list = violation_result["sum_of_eigenvalues"]["less_than_zero"]
fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.0001, annotation_vlines=[0])

# TODO: iterationsとnum_dataを与える方法
n_rep = len(violation_result["sorted_eigenvalues_list"][0])
n = num_data[0]
title = f"N={n}, Nrep={n_rep}, Number of Unphysical estimates={len(value_list)}"
fig.update_layout(title=title)
fig.update_xaxes( title=f"Sum of unphysical eigenvalues (<0)")
fig.show()

value_list = violation_result["sum_of_eigenvalues"]["greater_than_one"]
fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.0001, annotation_vlines=[1])

n_rep = len(violation_result["sorted_eigenvalues_list"][0])
n = num_data[0]
title = f"N={n}, Nrep={n_rep}, Number of Unphysical estimates={len(value_list)}"
fig.update_layout(title=title)  # TODO
fig.update_xaxes( title=f"Sum of unphysical eigenvalues (>1)")
fig.show()

## 内部状態

In [136]:
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

vec = np.array([1/np.sqrt(2), 0, 0, 0],dtype=np.float64)
true_object = State(c_sys, vec)
num_data = [1000]  # N
iterations = 1000  # Nrep

In [142]:
true_object.to_density_matrix()

array([[0.5+0.j, 0. +0.j],
       [0. +0.j, 0.5+0.j]])

In [48]:
povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
povms = [povm_x, povm_y, povm_z]

In [49]:
name = "1/np.sqrt(2), 0, 0, 0"
param_affine_est_linear = estimate(name, true_object, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate(name, true_object, num_data, iterations, on_para_eq_constraint=False)

param_affine_est_linear = [p[0] for p in param_affine_est_linear]
param_linear_est_linear = [p[0] for p in param_linear_est_linear]

  1%|          | 9/1000 [00:00<00:11, 86.36it/s]

time(s)=10.958998680114746


100%|██████████| 1000/1000 [00:11<00:00, 87.49it/s]

time(s)=11.434072256088257


## on_para_eq_constraint = True

In [50]:
violation_result = physicality_violation_check.get_physicality_violation_result_for_state(param_affine_est_linear)
violation_result.keys()

dict_keys(['violation_list'])

In [51]:
value_list = violation_result["violation_list"]
physicality_violation_check.make_prob_dist_histogram(value_list, x_range=(-1, 2), bin_size=0.01)

## on_para_eq_constraint = False

In [52]:
violation_result = physicality_violation_check.get_physicality_violation_result_for_state(param_linear_est_linear)
violation_result.keys()

100%|██████████| 1000/1000 [00:00<00:00, 7903.51it/s]


dict_keys(['sorted_eigenvalues_list', 'sum_of_eigenvalues'])

In [53]:
for i, values in enumerate(violation_result["sorted_eigenvalues_list"]):
    fig = physicality_violation_check.make_prob_dist_histogram(values, bin_size=0.0001)
    fig.update_layout(title=f"Nrep={len(values)}")
    fig.update_xaxes( title=f"Eigenvalue (i={i})")
    fig.show()

In [54]:
value_list = violation_result["sum_of_eigenvalues"]["less_than_zero"]
fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.0001, annotation_vlines=[0])

# TODO: iterationsとnum_dataを与える方法
n_rep = len(violation_result["sorted_eigenvalues_list"][0])
n = num_data[0]
title = f"N={n}, Nrep={n_rep}, Number of Unphysical estimates={len(value_list)}"
fig.update_layout(title=title)
fig.update_xaxes( title=f"Sum of unphysical eigenvalues (<0)")
fig.show()

value_list = violation_result["sum_of_eigenvalues"]["greater_than_one"]
fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.0001, annotation_vlines=[1])

n_rep = len(violation_result["sorted_eigenvalues_list"][0])
n = num_data[0]
title = f"N={n}, Nrep={n_rep}, Number of Unphysical estimates={len(value_list)}"
fig.update_layout(title=title)  # TODO
fig.update_xaxes( title=f"Sum of unphysical eigenvalues (>1)")
fig.show()